In [1]:
from supabase import create_client
from langchain_community.embeddings import OpenAIEmbeddings
import os
from dotenv import load_dotenv

load_dotenv()

# Initialize your Supabase client here
url = os.getenv("SUPABASE_URL")
key = os.getenv("SUPABASE_KEY")
supabase_client = create_client(url, key)

In [2]:
def get_all_document_ids_from_chunks():
    """
    Get all unique document_ids from the metadata of chunks_v2 table

    Returns:
        list: A list of unique document IDs
    """
    try:
        limit = 1000  # Batch size for pagination
        document_ids = set()  # Using set to ensure uniqueness
        
        # First get the total count of chunks
        count_response = supabase_client.table('chunks_v2').select('*', count='exact').limit(1).execute()
        total_count = count_response.count if count_response.count else 0
        
        # Loop through all pages
        for start in range(0, total_count, limit):
            end = start + limit - 1  # Calculate the end index
            try:
                response = supabase_client.table('chunks_v2') \
                    .select('metadata->document_id') \
                    .range(start, end) \
                    .execute()
                
                # Extract document_ids from the response and add to set
                for chunk in response.data:
                    if chunk.get('document_id'):  # Ensure we have a document_id
                        document_ids.add(chunk['document_id'])
                
            except Exception as e:
                print(f"Error fetching range {start}-{end}: {e}")
                continue
        
        return list(document_ids)  # Convert set back to list for return
        
    except Exception as e:
        print(f"Error fetching document IDs: {e}")
        return []

In [3]:
chunks_document_ids = get_all_document_ids_from_chunks()

# Print the results
print(f"Found {len(chunks_document_ids)} unique document IDs")
# for doc_id in chunks_document_ids:
#     print(doc_id)

Found 12587 unique document IDs


In [6]:
def get_all_ids_from_documents():
    """
    Get all IDs from the documents table

    Returns:
        list: A list of document IDs
    """
    try:
        limit = 1000  # Batch size for pagination
        document_ids = []
        
        # First get the total count
        count_response = supabase_client.table('documents').select('*', count='exact').limit(1).execute()
        total_count = count_response.count if count_response.count else 0
        
        # Loop through all pages
        for start in range(0, total_count, limit):
            end = start + limit - 1  # Calculate the end index
            try:
                response = supabase_client.table('documents') \
                    .select('id') \
                    .range(start, end) \
                    .execute()
                
                # Extract ids from the response
                batch_ids = [doc['id'] for doc in response.data]
                document_ids.extend(batch_ids)
                
            except Exception as e:
                print(f"Error fetching range {start}-{end}: {e}")
                continue
        
        return document_ids
        
    except Exception as e:
        print(f"Error fetching IDs from documents: {e}")
        return []

In [7]:
document_ids = get_all_ids_from_documents()
print(f"Found {len(document_ids)} documents")

Found 12629 documents


In [8]:
# check if all document_ids from chunks are in documents
missing_ids = set(chunks_document_ids) - set(document_ids)

print(f"Missing IDs: {missing_ids} and length: {len(missing_ids)}")

Missing IDs: {'34be4194-bb6f-491d-ae1b-d6c225dd44dd', '97c5a655-4370-4ba8-a7f2-740b9c1aca47', '598ea37d-a232-4b31-9bc4-7048a8160855', 'e5ebd2c5-140c-468e-bf6b-62c0689f20e0', '0492e2cd-2761-4b97-b3cf-8c4de1c84afa', 'a98c9209-3c08-4e8d-8daf-ed02b1019f6e', '90c0872b-7cc9-4b16-9fa7-624087e0b232', '902894a5-1ec8-46cd-bb06-21a9d95ce107'} and length: 8


In [4]:
from typing import Optional, Dict, List, Tuple

def get_document_company_mapping() -> Dict[str, str]:
    """
    Get a mapping of document_id to company_id from documents table
    
    Returns:
        Dict[str, str]: Dictionary mapping document_id to company_id
    """
    try:
        mapping = {}
        limit = 1000
        
        # Get total count
        count_response = supabase_client.table('documents').select('*', count='exact').limit(1).execute()
        total_count = count_response.count if count_response.count else 0
        
        # Fetch all documents with their company_ids
        for start in range(0, total_count, limit):
            end = start + limit - 1
            response = supabase_client.table('documents') \
                .select('id, company_id') \
                .range(start, end) \
                .execute()
            
            # Build mapping
            for doc in response.data:
                if doc.get('id') and doc.get('company_id'):
                    mapping[doc['id']] = doc['company_id']
        
        return mapping
    except Exception as e:
        print(f"Error getting document-company mapping: {e}")
        return {}

def update_chunks_metadata_with_company_id() -> Tuple[int, int, int]:
    """
    Update the metadata of chunks_v2 to include company_id and remove invalid document_ids
    
    Returns:
        tuple: (success_count, error_count, removed_count)
    """
    try:
        # Get document to company mapping
        doc_company_mapping = get_document_company_mapping()
        if not doc_company_mapping:
            print("No document-company mappings found")
            return 0, 0, 0
        
        success_count = 0
        error_count = 0
        removed_count = 0
        limit = 1000
        
        # Get total count of chunks
        count_response = supabase_client.table('chunks_v2').select('*', count='exact').limit(1).execute()
        total_count = count_response.count if count_response.count else 0
        
        # Process chunks in batches
        for start in range(0, total_count, limit):
            try:
                # Fetch batch of chunks
                response = supabase_client.table('chunks_v2') \
                    .select('id, metadata') \
                    .range(start, start + limit - 1) \
                    .execute()
                
                # Process each chunk in the batch
                for chunk in response.data:
                    try:
                        metadata = chunk.get('metadata', {})
                        doc_id = metadata.get('document_id')
                        
                        # Skip if chunk already has company_id
                        if metadata.get('company_id'):
                            continue
                            
                        # If document_id doesn't exist in mapping, remove the chunk
                        if doc_id and doc_id not in doc_company_mapping:
                            try:
                                delete_response = supabase_client.table('chunks_v2') \
                                    .delete() \
                                    .eq('id', chunk['id']) \
                                    .execute()
                                    
                                if delete_response.data:
                                    removed_count += 1
                                    print(f"Removed chunk {chunk['id']} with invalid document_id: {doc_id}")
                                continue
                            except Exception as delete_error:
                                print(f"Error deleting chunk {chunk['id']}: {delete_error}")
                                error_count += 1
                                continue
                        
                        # Get company_id from mapping
                        company_id = doc_company_mapping.get(doc_id)
                        if not company_id:
                            continue
                        
                        # Update metadata with company_id
                        metadata['company_id'] = company_id
                        
                        # Update the chunk in database
                        update_response = supabase_client.table('chunks_v2') \
                            .update({'metadata': metadata}) \
                            .eq('id', chunk['id']) \
                            .execute()
                        
                        if update_response.data:
                            success_count += 1
                        else:
                            error_count += 1
                            
                    except Exception as chunk_error:
                        print(f"Error processing chunk {chunk.get('id')}: {chunk_error}")
                        error_count += 1
                        continue
                
                print(f"Processed batch {start}-{start+limit}:")
                print(f"- Successful updates: {success_count}")
                print(f"- Removed chunks: {removed_count}")
                print(f"- Errors: {error_count}")
                
            except Exception as batch_error:
                print(f"Error processing batch {start}-{start+limit}: {batch_error}")
                continue
        
        return success_count, error_count, removed_count
    
    except Exception as e:
        print(f"Error updating chunks metadata: {e}")
        return 0, 0, 0

In [7]:
# Update the metadata and remove invalid chunks
success, errors, removed = update_chunks_metadata_with_company_id()
print("\nUpdate Summary:")
print(f"Successful updates: {success}")
print(f"Chunks removed: {removed}")
print(f"Errors encountered: {errors}")

# # Verify the update
# stats = verify_metadata_update()
# print("\nCurrent Status:")
# print(f"Total chunks: {stats['total_chunks']}")
# print(f"Chunks with company_id: {stats['with_company_id']}")
# print(f"Chunks without company_id: {stats['without_company_id']}")
# print(f"Chunks with invalid document_ids: {stats['invalid_document_ids']}")
# print(f"Completion percentage: {stats['completion_percentage']}%")

Processed batch 0-1000:
- Successful updates: 1
- Removed chunks: 0
- Errors: 0
Processed batch 1000-2000:
- Successful updates: 3
- Removed chunks: 0
- Errors: 0
Processed batch 2000-3000:
- Successful updates: 4
- Removed chunks: 0
- Errors: 0
Processed batch 3000-4000:
- Successful updates: 7
- Removed chunks: 0
- Errors: 0
Processed batch 4000-5000:
- Successful updates: 8
- Removed chunks: 0
- Errors: 0
Processed batch 5000-6000:
- Successful updates: 11
- Removed chunks: 0
- Errors: 0
Processed batch 6000-7000:
- Successful updates: 13
- Removed chunks: 0
- Errors: 0
Processed batch 7000-8000:
- Successful updates: 14
- Removed chunks: 0
- Errors: 0
Processed batch 8000-9000:
- Successful updates: 18
- Removed chunks: 0
- Errors: 0
Processed batch 9000-10000:
- Successful updates: 28
- Removed chunks: 0
- Errors: 0
Processed batch 10000-11000:
- Successful updates: 41
- Removed chunks: 0
- Errors: 0
Processed batch 11000-12000:
- Successful updates: 53
- Removed chunks: 0
- Error

In [5]:
def verify_metadata_update() -> Dict[str, int]:
    """
    Verify the metadata update by counting chunks with and without company_id
    
    Returns:
        Dict[str, int]: Statistics about the metadata update
    """
    try:
        # Count total chunks
        total_response = supabase_client.table('chunks_v2') \
            .select('*', count='exact') \
            .execute()
        total_chunks = total_response.count
        
        # Count chunks with company_id
        with_company_response = supabase_client.table('chunks_v2') \
            .select('*', count='exact') \
            .filter('metadata->>company_id', 'not.is', 'null') \
            .execute()
        with_company_id = with_company_response.count
        
        # Count chunks with document_id but without company_id
        invalid_doc_response = supabase_client.table('chunks_v2') \
            .select('*', count='exact') \
            .filter('metadata->>document_id', 'not.is', 'null') \
            .filter('metadata->>company_id', 'is', 'null') \
            .execute()
        invalid_doc_ids = invalid_doc_response.count
        
        # Calculate chunks without company_id
        without_company_id = total_chunks - with_company_id if total_chunks else 0
        
        return {
            'total_chunks': total_chunks,
            'with_company_id': with_company_id,
            'without_company_id': without_company_id,
            'invalid_document_ids': invalid_doc_ids,
            'completion_percentage': round((with_company_id / total_chunks * 100), 2) if total_chunks > 0 else 0
        }
        
    except Exception as e:
        print(f"Error verifying metadata update: {e}")
        import traceback
        print("Traceback:", traceback.format_exc())
        return {
            'total_chunks': 0,
            'with_company_id': 0,
            'without_company_id': 0,
            'invalid_document_ids': 0,
            'completion_percentage': 0
        }

In [8]:
# Verify the update
stats = verify_metadata_update()
print("\nCurrent Status:")
print(f"Total chunks: {stats['total_chunks']}")
print(f"Chunks with company_id: {stats['with_company_id']}")
print(f"Chunks without company_id: {stats['without_company_id']}")
print(f"Chunks with invalid document_ids: {stats['invalid_document_ids']}")
print(f"Completion percentage: {stats['completion_percentage']}%")


Current Status:
Total chunks: 21332
Chunks with company_id: 21300
Chunks without company_id: 32
Chunks with invalid document_ids: 32
Completion percentage: 99.85%


In [11]:
from datetime import datetime, timedelta
import requests
from collections import defaultdict

def fetch_bse_data(
    pageno: str = "1",
    strCat: str = "Company Update",
    strPrevDate: Optional[str] = None,
    strScrip: str = "538734",
    strSearch: str = "P",
    strToDate: Optional[str] = None,
    strType: str = "C",
    subcategory: str = "-1"
) -> list:
    """
    Fetch data from the BSE API using the specified parameters.
    Defaults are used if values are not provided by the user.
    
    Parameters:
    - pageno: The page number for pagination (default: "1").
    - strCat: Category of the data, e.g., 'Board Meeting', 'AGM/EGM', 'Result' (default: 'Board Meeting').
    - strPrevDate: Starting date of the range in YYYYMMDD format (default: "20240718").
    - strScrip: Company Scrip Code (default: "532215").
    - strSearch: Search type (default: "P").
    - strToDate: End date of the range in YYYYMMDD format (default: "20241018").
    - strType: Type of the announcement (default: "C").
    - subcategory: Subcategory ID (default: "-1").
    - base_url: The base URL for the API request (default: BSE announcement API URL).

    Returns:
    - list of information if the request is successful.
    - Error message if the request fails.
    """
    category_subcategory_map = {
        "AGM/EGM": ["AGM"],
        "Board Meeting": ["Board Meeting", "Outcome of Board Meeting"],
        "Corp. Action": ["Sub-division/Stock Split", "Record Date", "Dividend", "Bonds/Right Issue", "Bonus"],
        "Result": ["Financial Results"],
        "Company Update": [
            "General",
            "Acquisition",
            "Allotment of Equity Shares",
            "Allotment of Warrants",
            "Award of Order / Receipt of Order",
            "Buy back",
            "Delisting",
            "Joint Venture",
            "Open Offer",
            "Debt Securities",
            "Credit Rating",
            "Shareholding",
            "Appointment of Director",
            "Appointment of Chairman",
            "Appointment of Chairman and Managing Director",
            "Appointment of Chief Executive Officer (CEO)",
            "Appointment of Chief Financial Officer (CFO)",
            "Appointment of Managing Director & CEO",
            "Increase of Authorised Capital",
            "Board Meeting Deferred",
            "Board Meeting Postponed",
            "Board Meeting Rescheduled",
            "Board Meeting Cancelled",
            "Resignation of Director",
            "Resignation of Chairman",
            "Resignation of Chairman and Managing Director",
            "Resignation of Chief Executive Officer (CEO)",
            "Resignation of Chief Financial Officer (CFO)",
            "Resignation of Managing Director",
            "Investor Presentation",
            "Change in Management",
            "Issue of Securities",
            "Offer for Sale",
            "Open Offer - Updates",
            "Post Buyback Public Announcement",
            "Post Offer Public Announcement",
            "Preferential Issue",
            "Qualified Institutional Placement",
            "Raising of Funds",
            "Restructuring",
            "Earnings Call Transcript",
            "Public Announcement",
            "Revision of outcome",
            "Funds raising by issuance of Debt Securities by Large Entities",
            "Amalgamation/ Merger",
            "De-merger",
            "Acquisition",
            "Update-Acquisition/Scheme/Sale/Disposal/Reg30",
            "Change in Directors/ Key Managerial Personnel/ Auditor/ Compliance Officer/ Share Transfer Agent",
        ]

        # "Company Update": ["General", "Award of Order/Receipt of Order"]
    }

    base_url = "https://api.bseindia.com/BseIndiaAPI/api/AnnSubCategoryGetData/w"

    # Get today's date and 1 year prior in the format YYYYMMDD
    today = datetime.now().strftime("%Y%m%d")
    if strCat == "Result":
        d = 730
    else:
        d = 364
    from_ = (datetime.now() - timedelta(days=d)).strftime("%Y%m%d")

    # Use provided date or default to calculated dates
    strToDate = strToDate if strToDate else today
    strPrevDate = strPrevDate if strPrevDate else from_
    
    params = {
        "pageno": pageno,
        "strCat": strCat,
        "strPrevDate": strPrevDate,
        "strScrip": strScrip,
        "strSearch": strSearch,
        "strToDate": strToDate,
        "strType": strType,
        "subcategory": subcategory
    }
    
    headers = {
        "referer": "https://www.bseindia.com/",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36"
    }
    
    response = requests.get(base_url, params=params, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        valid_subcategories = category_subcategory_map.get(strCat, [])
        
        if valid_subcategories:
            # Group items by subcategory
            subcategory_groups = defaultdict(list)
            for item in data["Table"]:
                subcategory_name = item.get("SUBCATNAME", "").strip()
                if subcategory_name in valid_subcategories:
                    subcategory_groups[subcategory_name].append(item)
            
            # Take top 2 items from each subcategory
            filtered_data = []
            for subcategory_items in subcategory_groups.values():
                filtered_data.extend(subcategory_items[:2])
            
            return filtered_data
        
        return data["Table"][:2]
    
    return []

In [12]:


def update_document_subcategories(scrip_code: str, supabase_client) -> None:
    """
    Update subcategories for existing documents in the database.
    
    Parameters:
    - scrip_code: The BSE scrip code for the company
    - supabase_client: Supabase client instance
    """
    # Fetch all documents for this scrip code
    documents = supabase_client.table('documents')\
        .select('id, pdf_name, category_name, upload_date')\
        .eq('company_code', scrip_code)\
        .is_('subcategory_name', 'null')\
        .execute()
    
    if not documents.data:
        print(f"No documents found for scrip code {scrip_code} without subcategories")
        return
    
    print(f"Found {len(documents.data)} documents to update")
    
    for doc in documents.data:
        category_name = doc['category_name']
        pdf_name = doc['pdf_name']
        doc_id = doc['id']
        # upload_date = doc['upload_date']
        
        # # Convert upload_date to required format for BSE API
        # upload_dt = datetime.strptime(upload_date, '%Y-%m-%d')
        # from_date = (upload_dt - timedelta(days=364)).strftime('%Y%m%d')
        # to_date = (upload_dt + timedelta(days=1)).strftime('%Y%m%d')
        
        # Fetch data from BSE API for this specific date range and category
        fetched_data = fetch_bse_data(
            strCat=category_name,
            strScrip=scrip_code,
        )
        
        # Find matching document and get its subcategory
        subcategory_name = None
        for item in fetched_data:
            if item.get("ATTACHMENTNAME") == pdf_name:
                subcategory_name = item.get("SUBCATNAME", "").strip()
                break
        
        if subcategory_name:
            # Update the document with the subcategory
            supabase_client.table('documents')\
                .update({"subcategory_name": subcategory_name})\
                .eq('id', doc_id)\
                .execute()
            print(f"Updated document {pdf_name} with subcategory: {subcategory_name}")
        else:
            print(f"Could not find subcategory for document {pdf_name}")

In [20]:
update_document_subcategories("507685", supabase_client)

Found 8 documents to update
Updated document 60e5169e-2f58-4ead-8c35-b0126551eef3.pdf with subcategory: Outcome of Board Meeting
Updated document 26de1531-8266-42e3-9b2e-602d1b70edc2.pdf with subcategory: Board Meeting
Updated document 6a8df15b-8051-42dc-89c1-f4f50eccc895.pdf with subcategory: AGM
Updated document 50dbf26d-6405-4120-9c12-3438febcd4b2.pdf with subcategory: AGM
Updated document 64b0aa1f-f346-4dbf-a397-d1b722e71255.pdf with subcategory: Financial Results
Updated document c0bffcde-52a6-4b89-8e57-b65f3df9274c.pdf with subcategory: Financial Results
Updated document 4cc8c6b2-aa90-4745-a813-9b8ecb97f9fa.pdf with subcategory: General
Updated document b8664611-9fd1-4127-a9af-9c2910d7cd83.pdf with subcategory: General


In [15]:
supabase_client.table('documents')\
    .update({"subcategory_name": "test"})\
    .eq('id', '0664da95-a810-4f17-95e1-569ba666dce7')\
    .execute()

APIResponse[~_ReturnT](data=[{'id': '0664da95-a810-4f17-95e1-569ba666dce7', 'company_code': '538734', 'pdf_name': 'dadf95c8-5ca8-4d4a-ad47-697d7380baf9.pdf', 'category_name': 'Company Update', 'upload_date': '2024-10-09', 'company_id': 'CEINSYSTECH', 'subcategory_name': 'test'}], count=None)

In [24]:
page_size = 1000
start = 0
all_documents = []

while True:
    # Get documents with pagination
    result = supabase_client.table('documents')\
        .select('id, pdf_name, category_name, company_code')\
        .is_('subcategory_name', 'null')\
        .range(start, start + page_size - 1)\
        .execute()
    
    if not result.data:
        break
        
    all_documents.extend(result.data)
    
    if len(result.data) < page_size:
        break
        
    start += page_size

total_documents = len(all_documents)
print(f"Found {total_documents} documents with null subcategories")

Found 15794 documents with null subcategories


In [26]:
unique_codes = set(doc['company_code'] for doc in all_documents)

In [29]:
for scrip_code in unique_codes:
    print(f"\nProcessing company with scrip code: {scrip_code}")
    update_document_subcategories(scrip_code, supabase_client)


Processing company with scrip code: 544003
No documents found for scrip code 544003 without subcategories

Processing company with scrip code: 544250
No documents found for scrip code 544250 without subcategories

Processing company with scrip code: 512070
No documents found for scrip code 512070 without subcategories

Processing company with scrip code: 500027
Found 1 documents to update
Could not find subcategory for document 9431fc2f-e124-492b-b646-9875fe5ae302.pdf

Processing company with scrip code: 532400
No documents found for scrip code 532400 without subcategories

Processing company with scrip code: 530699
No documents found for scrip code 530699 without subcategories

Processing company with scrip code: 503101
No documents found for scrip code 503101 without subcategories

Processing company with scrip code: 505737
No documents found for scrip code 505737 without subcategories

Processing company with scrip code: 544131
No documents found for scrip code 544131 without subca

ConnectTimeout: HTTPSConnectionPool(host='api.bseindia.com', port=443): Max retries exceeded with url: /BseIndiaAPI/api/AnnSubCategoryGetData/w?pageno=1&strCat=Board+Meeting&strPrevDate=20231123&strScrip=532508&strSearch=P&strToDate=20241121&strType=C&subcategory=-1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001FB388866E0>, 'Connection to api.bseindia.com timed out. (connect timeout=None)'))

In [32]:
import time

def fetch_bse_data_with_retry(
    strCat: str,
    strScrip: str,
    max_retries: int = 3,
    timeout: int = 30,
    delay: int = 5
) -> list:
    """
    Fetch BSE data with retry logic and timeout handling.
    
    Parameters:
    - strCat: Category of data to fetch
    - strScrip: Company scrip code
    - max_retries: Maximum number of retry attempts
    - timeout: Timeout in seconds for each request
    - delay: Delay in seconds between retries
    """
    base_url = "https://api.bseindia.com/BseIndiaAPI/api/AnnSubCategoryGetData/w"

    # Get today's date and 1 year prior
    today = datetime.now().strftime("%Y%m%d")
    if strCat == "Result":
        d = 730
    else:
        d = 364
    from_ = (datetime.now() - timedelta(days=d)).strftime("%Y%m%d")
    
    params = {
        "pageno": "1",
        "strCat": strCat,
        "strPrevDate": from_,
        "strScrip": strScrip,
        "strSearch": "P",
        "strToDate": today,
        "strType": "C",
        "subcategory": "-1"
    }
    
    headers = {
        "referer": "https://www.bseindia.com/",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36"
    }

    for attempt in range(max_retries):
        try:
            response = requests.get(
                base_url, 
                params=params, 
                headers=headers,
                timeout=timeout
            )
            
            if response.status_code == 200:
                data = response.json()
                return data.get("Table", [])[:2]
            
        except (requests.ConnectionError, requests.Timeout) as e:
            print(f"Attempt {attempt + 1} failed: {str(e)}")
            if attempt < max_retries - 1:
                print(f"Waiting {delay} seconds before retrying...")
                time.sleep(delay)
            continue
            
    print(f"Failed to fetch data after {max_retries} attempts")
    return []

def update_all_null_subcategories():
    """
    Get all documents with null subcategories using pagination and process them one by one.
    """
    # supabase_client = create_supabase_client()
    
    # Initialize variables for pagination
    page_size = 1000
    start = 0
    all_documents = []
    
    while True:
        # Get documents with pagination
        result = supabase_client.table('documents')\
            .select('id, pdf_name, category_name, company_code')\
            .is_('subcategory_name', 'null')\
            .range(start, start + page_size - 1)\
            .execute()
        
        if not result.data:
            break
            
        all_documents.extend(result.data)
        
        if len(result.data) < page_size:
            break
            
        start += page_size
    
    # Group documents by company code
    company_docs = {}
    for doc in all_documents:
        company_code = doc['company_code']
        if company_code not in company_docs:
            company_docs[company_code] = []
        company_docs[company_code].append(doc)
    
    total_companies = len(company_docs)
    print(f"Found {len(all_documents)} documents across {total_companies} companies")
    
    # Process each company's documents
    for idx, (company_code, docs) in enumerate(company_docs.items(), 1):
        print(f"\nProcessing company {idx}/{total_companies} with scrip code: {company_code}")
        
        # Group documents by category
        category_docs = {}
        for doc in docs:
            category = doc['category_name']
            if category not in category_docs:
                category_docs[category] = []
            category_docs[category].append(doc)
        
        # Process each category
        for category, category_docs_list in category_docs.items():
            print(f"Fetching data for category: {category}")
            
            # Get data from BSE API with retry logic
            fetched_data = fetch_bse_data_with_retry(
                strCat=category,
                strScrip=company_code
            )
            
            # Create mapping of pdf_name to subcategory
            pdf_subcategory_map = {
                item["ATTACHMENTNAME"]: item.get("SUBCATNAME", "").strip()
                for item in fetched_data
            }
            
            # Update documents
            for doc in category_docs_list:
                subcategory = pdf_subcategory_map.get(doc['pdf_name'])
                if subcategory:
                    try:
                        supabase_client.table('documents')\
                            .update({"subcategory_name": subcategory})\
                            .eq('id', doc['id'])\
                            .execute()
                        print(f"Updated document {doc['pdf_name']} with subcategory: {subcategory}")
                    except Exception as e:
                        print(f"Error updating document {doc['pdf_name']}: {str(e)}")
                else:
                    print(f"Could not find subcategory for document {doc['pdf_name']}")
        
        # Add a small delay between companies to avoid rate limiting
        time.sleep(2)

# Usage:
# update_all_null_subcategories()

In [33]:
update_all_null_subcategories()

Found 12644 documents across 1557 companies

Processing company 1/1557 with scrip code: 543237
Fetching data for category: Board Meeting
Updated document 88b6849e-dcec-4719-ad2f-38b848059f38.pdf with subcategory: Board Meeting
Updated document 89c30338-afdb-4f89-914e-03a2864cf87d.pdf with subcategory: Outcome of Board Meeting
Fetching data for category: AGM/EGM
Updated document 463b9244-7e5e-4d81-abf2-af8ca3146aa5.pdf with subcategory: AGM
Could not find subcategory for document 7e29cf4d-4afc-4349-8d16-e73fd9aa5293.pdf
Fetching data for category: Result
Updated document df9ca914-8be6-4b41-9332-ac5457c24f2e.pdf with subcategory: Financial Results
Updated document 3154380d-a851-4f1b-850a-2cfe6dd93fc2.pdf with subcategory: Financial Results

Processing company 2/1557 with scrip code: 532477
Fetching data for category: Board Meeting
Updated document 5d399273-1d01-406d-aa03-41cdf0c2b94c.pdf with subcategory: Outcome of Board Meeting
Updated document df83e01d-e618-4040-9414-f70eb6a1f439.pdf 